In [3]:
import numpy as np

import pandas as pd

import copy


from shapely.geometry import Point, Polygon


In [10]:
'''Load data'''

'''Zone-related data'''

Policy='Stay'

Taxi_Zones=np.load('./Data/NYC_Zones/Taxi_Zones.npy',allow_pickle=True).item()

Zone_list=np.load('./Data/NYC_Zones/Zone_list.npy',allow_pickle=True)

Zone_Center=np.load('./Data/NYC_Zones/Zone_Center.npy',allow_pickle=True).item()

Zone_Link=np.load('./Data/NYC_Zones/Zone_Link.npy',allow_pickle=True).item()

'''Zone matrix'''

Connect_matrix=np.load('./Data/NYC_Zones/Connect_matrix.npy',allow_pickle=True)


End_step=180


# State and Action

In [11]:

State=list()

for zone in Zone_list:
    
    for step in range(End_step):
        
        State.append(zone+'_'+str(step))
        
np.save('./Data/MDP/State',State)
        
Action={}

for state in State:
    
    zone_index=int(state.split('_')[1])
    
    candidates=np.argwhere(Connect_matrix[zone_index]==1.0)
    
    Action[state]=['Zone_'+str(i[0]) for i in candidates]
        
np.save('./Data/MDP/Action',Action)

# Estimate the Parameters: Matching Probability

In [12]:
def Get_prob(ratio):
        
    prob = 1- np.exp(-1*ratio)

    return prob

In [13]:
'''Available Orders'''

Order_CNT=pd.read_csv('./Data/NYC_Trips/Order_CNT.csv')

Order_CNT=Order_CNT.drop(columns=['Unnamed: 0'])

Order_CNT['step']=Order_CNT.apply(lambda x:int((x['Arrive_step']-2520)/6),axis=1)

Order_CNT=Order_CNT.drop_duplicates(subset=['step','Order_id'])

Order_CNT=Order_CNT.groupby(['step','Pickup_Zone']).count()

Order_CNT['Transition']=Order_CNT.index

Order_CNT['Step']=Order_CNT.apply(lambda x:x['Transition'][0],axis=1)

Order_CNT['Zone']=Order_CNT.apply(lambda x:x['Transition'][1],axis=1)

Order_CNT=Order_CNT.reset_index(drop=True)

Order_CNT=Order_CNT.rename(index=str, columns={'Order_id': 'Order_Cnt'})

Order_CNT=Order_CNT[['Step','Zone','Order_Cnt']]

'''Idle Drivers'''

Driver_df=pd.read_csv('./Data/NYC_Trips/Dispatched_Driver_df_'+Policy+'.csv')

Driver_df=Driver_df.drop(columns=['Unnamed: 0'])

Driver_CNT=Driver_df.loc[Driver_df['Step']<=3600]

Driver_CNT['step']=Driver_CNT.apply(lambda x:int((x['Step']-2520)/6),axis=1)

Driver_CNT=Driver_CNT.drop_duplicates(subset=['step','Driver_id'])

Driver_CNT=Driver_CNT.groupby(['step','Zone']).count()

Driver_CNT['Transition']=Driver_CNT.index

Driver_CNT['Step']=Driver_CNT.apply(lambda x:x['Transition'][0],axis=1)

Driver_CNT['Zone']=Driver_CNT.apply(lambda x:x['Transition'][1],axis=1)

Driver_CNT=Driver_CNT.reset_index(drop=True)

Driver_CNT=Driver_CNT.rename(index=str, columns={'Driver_id': 'Driver_Cnt'})

Driver_CNT=Driver_CNT[['Step','Zone','Driver_Cnt']]

'''Overall Demand supply'''

Demand_supply=Driver_CNT.merge(Order_CNT,on=['Step','Zone'],how='left')

Demand_supply=Demand_supply.fillna(0)

Demand_supply['ratio']=Demand_supply.apply(lambda x:x['Order_Cnt']/x['Driver_Cnt'],axis=1)

Demand_supply['Prob']=Demand_supply.apply(lambda x:Get_prob(x['ratio']),axis=1)

Demand_supply


'''Matching Prob'''

Matching_Prob={s:0.0 for s in State}

for idx,row in Demand_supply.iterrows():
    
    state=row['Zone']+'_'+str(int(row['Step']))
    
    Matching_Prob[state]=row['Prob']
    
    


np.save('./Data/MDP/Matching_Prob',Matching_Prob)

/Users/yueyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Estimate the Parameters: Pickup Probability

In [14]:
'''Available Orders'''

Order_CNT=pd.read_csv('./Data/NYC_Trips/Order_CNT.csv')

Order_CNT=Order_CNT.drop(columns=['Unnamed: 0'])

Driver_CNT=copy.copy(Driver_df.loc[(Driver_df['Step']<=3600)& (Driver_df['Order_id']!='Idle')])

Driver_CNT['step']=Driver_CNT.apply(lambda x:int((x['Step']-2520)/6),axis=1)

Pickup_df=Driver_CNT.merge(Order_CNT[['Order_id','Response_step','Pickup_Zone']],on=['Order_id'],how='left')

Pickup_df=Pickup_df[['step','Zone','Pickup_Zone','Driver_id']]

Pickup_CNT=Pickup_df.groupby(by=['Zone','Pickup_Zone']).count()

Pickup_CNT['Transition']=Pickup_CNT.index

Pickup_CNT['Zone']=Pickup_CNT.apply(lambda x:x['Transition'][0],axis=1)

Pickup_CNT['Pickup_Zone']=Pickup_CNT.apply(lambda x:x['Transition'][1],axis=1)

Pickup_CNT=Pickup_CNT.reset_index(drop=True)

Pickup_CNT=Pickup_CNT.rename(index=str, columns={'Driver_id': 'Order_Num'})

Pickup_CNT=Pickup_CNT[['Zone','Pickup_Zone','Order_Num']]

Temp=Pickup_df.groupby(by=['Zone']).count()

Temp['Zone']=Temp.index

Temp=Temp.reset_index(drop=True)

Temp=Temp.rename(index=str, columns={'Driver_id': 'Sum'})

Temp=Temp[['Zone','Sum']]

Pickup_CNT=Pickup_CNT.merge(Temp,on=['Zone'],how='left')

Pickup_CNT['Prob']=Pickup_CNT.apply(lambda x:x['Order_Num']/x['Sum'],axis=1)



'''Pickup Prob'''

Pickup_Prob={z:{} for z in Zone_list}


for idx,row in Pickup_CNT.iterrows():
    
    zone=row['Zone']
    
    Pickup_Prob[zone][row['Pickup_Zone']]=row['Prob']


np.save('./Data/MDP/Pickup_Prob',Pickup_Prob)


# Estimate the Parameters: Destination Probability

In [15]:
Order_CNT=pd.read_csv('./Data/NYC_Trips/Order_CNT.csv')

Order_CNT=Order_CNT.drop(columns=['Unnamed: 0'])

Dest_CNT=Order_CNT.groupby(by=['Pickup_Zone','Dropoff_Zone']).count()

Dest_CNT['Transition']=Dest_CNT.index

Dest_CNT['Pickup_Zone']=Dest_CNT.apply(lambda x:x['Transition'][0],axis=1)

Dest_CNT['Dropoff_Zone']=Dest_CNT.apply(lambda x:x['Transition'][1],axis=1)

Dest_CNT=Dest_CNT.reset_index(drop=True)

Dest_CNT=Dest_CNT.rename(index=str, columns={'Driver_id': 'Order_Num'})

Dest_CNT=Dest_CNT[['Pickup_Zone','Dropoff_Zone','Order_Num']]

Temp=Order_CNT.groupby(by=['Pickup_Zone']).count()

Temp['Pickup_Zone']=Temp.index

Temp=Temp.reset_index(drop=True)

Temp=Temp.rename(index=str, columns={'Driver_id': 'Sum'})

Temp=Temp[['Pickup_Zone','Sum']]

Dest_CNT=Dest_CNT.merge(Temp,on=['Pickup_Zone'],how='left')

Dest_CNT['Prob']=Dest_CNT.apply(lambda x:x['Order_Num']/x['Sum'],axis=1)

Dest_CNT

'''Dest Prob'''

Dest_Prob={z:{} for z in Zone_list}


for idx,row in Dest_CNT.iterrows():
    
    zone=row['Pickup_Zone']
    
    Dest_Prob[zone][row['Dropoff_Zone']]=row['Prob']
    
    
np.save('./Data/MDP/Dest_Prob',Dest_Prob)







# Estimate the Parameters: Shortest Travel time

In [16]:
Travel_time={z:{z:0 for z in Zone_list} for z in Zone_list}

for z in Zone_list:
    for z_ in Zone_list:
        
        Travel_time[z][z_]=int((Point(Zone_Center[z]).distance( Point(Zone_Center[z_]))*1.3*111)/0.6)


np.save('./Data/MDP/Travel_time',Travel_time)

